In [2]:
# imports
import requests
import json
import numpy as np
import pandas as pd

base_url = 'https://api-web.nhle.com/v1/'
# API TEST
url = 'https://api-web.nhle.com/v1/roster/VAN/20242025'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print(f'Error: {response.status_code}')

print(data)

/Users/derekhuang/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


{'forwards': [{'id': 8476927, 'headshot': 'https://assets.nhle.com/mugs/nhl/20242025/VAN/8476927.png', 'firstName': {'default': 'Teddy', 'cs': 'Theodors', 'sk': 'Theodors'}, 'lastName': {'default': 'Blueger', 'cs': 'Blugers', 'sk': 'Blugers'}, 'sweaterNumber': 53, 'positionCode': 'C', 'shootsCatches': 'L', 'heightInInches': 72, 'weightInPounds': 185, 'heightInCentimeters': 183, 'weightInKilograms': 84, 'birthDate': '1994-08-15', 'birthCity': {'default': 'Riga', 'fi': 'Riika'}, 'birthCountry': 'LVA'}, {'id': 8478444, 'headshot': 'https://assets.nhle.com/mugs/nhl/20242025/VAN/8478444.png', 'firstName': {'default': 'Brock'}, 'lastName': {'default': 'Boeser'}, 'sweaterNumber': 6, 'positionCode': 'R', 'shootsCatches': 'R', 'heightInInches': 73, 'weightInPounds': 208, 'heightInCentimeters': 185, 'weightInKilograms': 94, 'birthDate': '1997-02-25', 'birthCity': {'default': 'Burnsville'}, 'birthCountry': 'USA', 'birthStateProvince': {'default': 'MN'}}, {'id': 8478498, 'headshot': 'https://asset

## Get all team data

In [3]:
url = "https://api.nhle.com/stats/rest/en/team/summary?cayenneExp=seasonId=20242025"

response = requests.get(url)

team_data = response.json()
team_ids = []
for team in team_data['data']:
    team_ids.append(team['teamId'])

print(team_ids)

[9, 59, 17, 8, 3, 12, 29, 30, 54, 18, 22, 16, 25, 24, 19, 28, 20, 1, 5, 2, 10, 7, 52, 4, 23, 21, 6, 14, 13, 55, 26, 15]


In [4]:
import requests

# URL to fetch data from current teams
current_teams_url = 'https://api.nhle.com/stats/rest/en/team/summary?cayenneExp=seasonId=20242025'

# Send GET request to NHL API
response = requests.get(current_teams_url)
current_team_data = response.json()

# Retrieve team IDs and abbreviations
team_ids = [team['teamId'] for team in current_team_data['data']]
#print("Team IDs:", team_ids)

teams_url = 'https://api.nhle.com/stats/rest/en/team'
team_data = requests.get(teams_url).json()
#print(team_data)
team_dict = {}


for data in team_data['data']:
    for id in team_ids:
        if data['id'] == id:
            team_dict[id] = data['triCode']

team_dict = dict(sorted(team_dict.items()))

## Get all players for each team
endpoint: /v1/roster/{team}/current

In [5]:
player_ids = []
team_count = 0
for item in team_dict:
    team_count += 1
    print(team_count)
    team_abbrv = team_dict[item]
    roster_url_endpoint = f'roster/{team_abbrv}/current'
    final_url = base_url + roster_url_endpoint

    print(final_url)
    response = requests.get(final_url)
    roster_data = response.json()
    
    for forward in roster_data['forwards']:
        player_ids.append(forward['id'])

    for defense in roster_data['defensemen']:
        player_ids.append(defense['id'])

    for goalie in roster_data['goalies']:
        player_ids.append(goalie['id'])

    print('current len: ', len(player_ids))

print(len(player_ids))

1
https://api-web.nhle.com/v1/roster/NJD/current
current len:  25
2
https://api-web.nhle.com/v1/roster/NYI/current
current len:  51
3
https://api-web.nhle.com/v1/roster/NYR/current
current len:  74
4
https://api-web.nhle.com/v1/roster/PHI/current
current len:  100
5
https://api-web.nhle.com/v1/roster/PIT/current
current len:  124
6
https://api-web.nhle.com/v1/roster/BOS/current
current len:  145
7
https://api-web.nhle.com/v1/roster/BUF/current
current len:  169
8
https://api-web.nhle.com/v1/roster/MTL/current
current len:  191
9
https://api-web.nhle.com/v1/roster/OTT/current
current len:  215
10
https://api-web.nhle.com/v1/roster/TOR/current
current len:  245
11
https://api-web.nhle.com/v1/roster/CAR/current
current len:  266
12
https://api-web.nhle.com/v1/roster/FLA/current
current len:  288
13
https://api-web.nhle.com/v1/roster/TBL/current
current len:  310
14
https://api-web.nhle.com/v1/roster/WSH/current
current len:  338
15
https://api-web.nhle.com/v1/roster/CHI/current
current le

populating a player csv table

In [6]:
player_table = pd.DataFrame()

for player_id in player_ids:
    try:
        player_endpoint = f'player/{player_id}/landing'
        final_url = base_url + player_endpoint
        response = requests.get(final_url)

        if response.status_code != 200:
            print(f"Skipping player ID {player_id}: Status code {response.status_code}")
            continue

        player_data = response.json()

        # Skip if required fields are missing
        if not all(key in player_data for key in ["playerId", "firstName", "lastName", "careerTotals"]):
            print(f"Missing data for player ID {player_id}")
            continue

        player_info = {
            "Player ID": player_data["playerId"],
            "First Name": player_data["firstName"].get("default", "N/A"),
            "Last Name": player_data["lastName"].get("default", "N/A"),
            "Team Abbreviation": player_data.get("currentTeamAbbrev", "N/A"),
            "Sweater Number": player_data.get("sweaterNumber", "N/A"),
            "Position": player_data.get("position", "N/A"),
            "Goals": player_data["careerTotals"]["regularSeason"].get("goals", 0),
            "Assists": player_data["careerTotals"]["regularSeason"].get("assists", 0),
            "Points": player_data["careerTotals"]["regularSeason"].get("points", 0),
            "Games Played": player_data["careerTotals"]["regularSeason"].get("gamesPlayed", 0),
            "Plus/Minus": player_data["careerTotals"]["regularSeason"].get("plusMinus", 0),
        }

        player_table = pd.concat([player_table, pd.DataFrame([player_info])], ignore_index=True)

    except KeyError as e:
        print(f"KeyError for player ID {player_id}: {e}")
        continue
    except Exception as e:
        print(f"Unexpected error for player ID {player_id}: {e}")
        continue


Missing data for player ID 8482131
Missing data for player ID 8482731
Missing data for player ID 8481707


In [7]:
player_table.to_csv('nhl_players.csv', index=False)

In [44]:
first_names = list(player_table['First Name'])
last_names = list(player_table['Last Name'])

names_formatted = []

for i in range(len(first_names)):
    names_formatted.append(last_names[i][0].lower() + '/' + last_names[i][:5].lower() + first_names[i][:2].lower()+'01.html')

print(names_formatted)
print(len(names_formatted))
names_formatted

['b/bastina01.html', 'b/brattje01.html', 'c/cottepa01.html', 'd/dowliju01.html', 'h/haulaer01.html', 'h/hischni01.html', 'h/hugheja01.html', 'l/lazarcu01.html', 'm/macdeku01.html', 'm/meierti01.html', 'm/merceda01.html', 'n/noesest01.html', 'p/palaton01.html', 't/tatarto01.html', 'd/dillobr01.html', 'h/hamildo01.html', 'h/hataksa01.html', 'h/hughelu01.html', 'k/kovacjo01.html', 'p/pescebr01.html', 's/siegejo01.html', 'a/allenja01.html', 'm/marksja01.html', 'b/barzama01.html', 'c/cizikca01.html', 'd/duclaan01.html', 'e/engvapi01.html', 'f/faschhu01.html', 'h/holmssi01.html', 'h/horvabo01.html', 'l/leean01.html', 'm/macleky01.html', 'm/martima01.html', 'n/nelsobr01.html', 'p/pageaje01.html', 'p/palmiky01.html', 't/tsyplma01.html', 'w/wahlsol01.html', 'c/cholode01.html', 'd/dobsono01.html', 'g/georgis01.html', 'h/huttogr01.html', 'm/mayfisc01.html', 'p/pelecad01.html', 'p/pulocry01.html', 'r/reillmi01.html', 'r/romanal01.html', 's/sorokil01.html', 'v/varlase01.html', 'b/brodzjo01.html', '

['b/bastina01.html',
 'b/brattje01.html',
 'c/cottepa01.html',
 'd/dowliju01.html',
 'h/haulaer01.html',
 'h/hischni01.html',
 'h/hugheja01.html',
 'l/lazarcu01.html',
 'm/macdeku01.html',
 'm/meierti01.html',
 'm/merceda01.html',
 'n/noesest01.html',
 'p/palaton01.html',
 't/tatarto01.html',
 'd/dillobr01.html',
 'h/hamildo01.html',
 'h/hataksa01.html',
 'h/hughelu01.html',
 'k/kovacjo01.html',
 'p/pescebr01.html',
 's/siegejo01.html',
 'a/allenja01.html',
 'm/marksja01.html',
 'b/barzama01.html',
 'c/cizikca01.html',
 'd/duclaan01.html',
 'e/engvapi01.html',
 'f/faschhu01.html',
 'h/holmssi01.html',
 'h/horvabo01.html',
 'l/leean01.html',
 'm/macleky01.html',
 'm/martima01.html',
 'n/nelsobr01.html',
 'p/pageaje01.html',
 'p/palmiky01.html',
 't/tsyplma01.html',
 'w/wahlsol01.html',
 'c/cholode01.html',
 'd/dobsono01.html',
 'g/georgis01.html',
 'h/huttogr01.html',
 'm/mayfisc01.html',
 'p/pelecad01.html',
 'p/pulocry01.html',
 'r/reillmi01.html',
 'r/romanal01.html',
 's/sorokil01.h

In [8]:
first_names = list(player_table['First Name'])
last_names = list(player_table['Last Name'])

names_formatted = []

for i in range(len(first_names)):
    names_formatted.append(first_names[i].lower() + '-' + last_names[i].lower())

print(names_formatted)
print(len(names_formatted))
names_formatted

['nathan-bastian', 'shane-bowers', 'jesper-bratt', 'paul-cotter', 'justin-dowling', 'erik-haula', 'nico-hischier', 'jack-hughes', 'curtis-lazar', 'kurtis-macdermid', 'timo-meier', 'dawson-mercer', 'stefan-noesen', 'ondrej-palat', 'tomas-tatar', 'nick-desimone', 'brenden-dillon', 'dougie-hamilton', 'santeri-hatakka', 'luke-hughes', 'johnathan-kovacevic', 'brett-pesce', 'jonas-siegenthaler', 'jake-allen', 'jacob-markstrom', 'mathew-barzal', 'casey-cizikas', 'anthony-duclair', 'pierre-engvall', 'hudson-fasching', 'simon-holmstrom', 'bo-horvat', 'anders-lee', 'kyle-maclean', 'matt-martin', 'brock-nelson', 'jean-gabriel-pageau', 'kyle-palmieri', 'maxim-tsyplakov', 'oliver-wahlstrom', 'dennis-cholowski', 'noah-dobson', 'isaiah-george', 'grant-hutton', 'scott-mayfield', 'adam-pelech', 'ryan-pulock', 'mike-reilly', 'alexander-romanov', 'ilya-sorokin', 'semyon-varlamov', 'brett-berard', 'jonny-brodzinski', 'sam-carrick', 'filip-chytil', 'will-cuylle', 'adam-edstrom', 'kaapo-kakko', 'chris-kreid

['nathan-bastian',
 'shane-bowers',
 'jesper-bratt',
 'paul-cotter',
 'justin-dowling',
 'erik-haula',
 'nico-hischier',
 'jack-hughes',
 'curtis-lazar',
 'kurtis-macdermid',
 'timo-meier',
 'dawson-mercer',
 'stefan-noesen',
 'ondrej-palat',
 'tomas-tatar',
 'nick-desimone',
 'brenden-dillon',
 'dougie-hamilton',
 'santeri-hatakka',
 'luke-hughes',
 'johnathan-kovacevic',
 'brett-pesce',
 'jonas-siegenthaler',
 'jake-allen',
 'jacob-markstrom',
 'mathew-barzal',
 'casey-cizikas',
 'anthony-duclair',
 'pierre-engvall',
 'hudson-fasching',
 'simon-holmstrom',
 'bo-horvat',
 'anders-lee',
 'kyle-maclean',
 'matt-martin',
 'brock-nelson',
 'jean-gabriel-pageau',
 'kyle-palmieri',
 'maxim-tsyplakov',
 'oliver-wahlstrom',
 'dennis-cholowski',
 'noah-dobson',
 'isaiah-george',
 'grant-hutton',
 'scott-mayfield',
 'adam-pelech',
 'ryan-pulock',
 'mike-reilly',
 'alexander-romanov',
 'ilya-sorokin',
 'semyon-varlamov',
 'brett-berard',
 'jonny-brodzinski',
 'sam-carrick',
 'filip-chytil',
 'wi

In [10]:
import requests
from bs4 import BeautifulSoup
import csv
import time

#base = 'https://www.hockey-reference.com/players/'
base = 'https://capwages.com/players/'
data = pd.DataFrame()
data['First Name'] = first_names
data['Last Name'] = last_names
data['Cap Hit'] = np.nan


# Loop through player paths
for i, path in enumerate(names_formatted):
    url = base + path
    print(f"Fetching URL: {url}")
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            print('Success: Page loaded.')
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract cap hit
            cap_hit = None
            for div in soup.find_all('div', class_='flex justify-between'):
                dt = div.find('dt', class_='text-sm text-muted-foreground')
                if dt and 'Cap Hit' in dt.text:
                    dd = div.find('dd', class_='text-sm font-medium text-right')
                    if dd:
                        # Find the specific span with the desired class
                        span = dd.find('span', class_='hidden sm:inline')
                        if span:
                            cap_hit = span.get_text(strip=True)
                            break

            
            # Update DataFrame
            data.at[i, 'Cap Hit'] = cap_hit if cap_hit else 'N/A'
        else:
            print(f"Error: Received status code {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch {url}: {e}")

data.head()

Fetching URL: https://capwages.com/players/nathan-bastian
Success: Page loaded.
Fetching URL: https://capwages.com/players/shane-bowers


/var/folders/_z/1h8m2g3x1cgc41srwzsqjr900000gn/T/ipykernel_22117/2412309310.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '$1,350,000' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.at[i, 'Cap Hit'] = cap_hit if cap_hit else 'N/A'


Success: Page loaded.
Fetching URL: https://capwages.com/players/jesper-bratt
Success: Page loaded.
Fetching URL: https://capwages.com/players/paul-cotter
Success: Page loaded.
Fetching URL: https://capwages.com/players/justin-dowling
Success: Page loaded.
Fetching URL: https://capwages.com/players/erik-haula
Success: Page loaded.
Fetching URL: https://capwages.com/players/nico-hischier
Success: Page loaded.
Fetching URL: https://capwages.com/players/jack-hughes
Success: Page loaded.
Fetching URL: https://capwages.com/players/curtis-lazar
Success: Page loaded.
Fetching URL: https://capwages.com/players/kurtis-macdermid
Success: Page loaded.
Fetching URL: https://capwages.com/players/timo-meier
Success: Page loaded.
Fetching URL: https://capwages.com/players/dawson-mercer
Success: Page loaded.
Fetching URL: https://capwages.com/players/stefan-noesen
Success: Page loaded.
Fetching URL: https://capwages.com/players/ondrej-palat
Success: Page loaded.
Fetching URL: https://capwages.com/play

,First Name,Last Name,Cap Hit
0,Nathan,Bastian,"$1,350,000"
1,Shane,Bowers,"$775,000"
2,Jesper,Bratt,"$7,875,000"
3,Paul,Cotter,"$775,000"
4,Justin,Dowling,"$775,000"


In [11]:
data.to_csv(r'24-25 player cap_hits.csv', index = False)